In [60]:
import pandas as pd
import numpy as np
import random
import math
from typing import Dict, List, Tuple

import plotly.express as px
pd.options.plotting.backend = "plotly"

In [61]:
#market behavior simulation functions
def short_sin(day:int, cycle:int):
    value = 1.5 + (0.5 * math.sin((day - 7.5) / (cycle / (2*math.pi))))
    return value


def short_cos(day:int, cycle:int):
    value = 1.5 + (0.5 * math.cos((day + cycle) / (cycle / (2*math.pi))))
    return value


def long_sin(day:int, cycle:int, offset:float):
    value = 1 + (0.5 * math.sin(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


def long_cos(day:int, cycle:int, offset:float):
    value = 1 + (0.5 * math.cos(((day + cycle * offset) % cycle) / (cycle / (2*math.pi))))
    return value


#reward rate framework
def rr_framework(supply:int):
    if supply > 1000000000:
      return 0.001
    elif supply > 100000000:
      return 0.002
    else:
      return 0.004

In [62]:
class ModelParams():
    def __init__(self, horizon:int, max_liq_ratio:float, max_liq_fmcap_ratio:float, cycle_reweights:float, release_capture:float, demand_factor:float, supply_factor:float, initial_supply:float, initial_reserves:float, initial_liq_usd:float, arb_factor:float, initial_price:float, initial_target:float, target_price_function:str, short_cycle:int, long_cycle:int, long_sin_offset:float, long_cos_offset:float):
        self.horizon = horizon
        self.max_liq_ratio = max_liq_ratio
        self.max_liq_fmcap_ratio = max_liq_fmcap_ratio
        self.cycle_reweights = cycle_reweights
        self.release_capture = release_capture
        self.demand_factor = demand_factor
        self.supply_factor = supply_factor
        self.short_cycle = short_cycle
        self.long_cycle = long_cycle
        self.long_sin_offset = long_sin_offset
        self.long_cos_offset = long_cos_offset

        self.initial_supply = initial_supply
        self.initial_reserves = initial_reserves
        self.initial_liq_usd = initial_liq_usd
        self.initial_price = initial_price
        self.initial_target = initial_target
        self.target_price_function = target_price_function
        self.arb_factor = arb_factor


class Day():
    def __init__(self, params:ModelParams, prev_arbs:Dict[int, Tuple[float, float]], prev_lags=Dict[int, Tuple[int, Dict[int, float]]], prev_day=None):

        if prev_day is None:
            self.day = 1
            self.supply = params.initial_supply
            self.reward_rate = rr_framework(self.supply)
            self.price = params.initial_price
            self.natural_target = params.initial_target
            self.real_target = params.initial_target
            self.liq_usd = params.initial_liq_usd
            self.liq_ohm = self.liq_usd / self.price
            self.k = (self.liq_usd ** 2) / self.price
            self.reserves_in = 0
            self.reserves_out = 0
            self.release_capture = 0
            self.reserves = params.initial_reserves
            self.market_demand = params.demand_factor
            self.market_supply = params.supply_factor
            self.arb_factor = params.arb_factor
            self.arb_demand = 0
            self.arb_supply = 0
            self.unwind_demand = 0
            self.unwind_supply = 0
            self.net_flow = random.uniform(self.liq_usd * self.market_supply, self.liq_usd * self.market_demand)
            prev_arbs[self.day] = (self.arb_demand, self.arb_supply)

        else:
            self.day = prev_day.day + 1
            self.reward_rate = rr_framework(prev_day.supply)
            self.supply = prev_day.supply * (1 + self.reward_rate)
            self.natural_target = calc_natural_target(params=params, prev_day=prev_day)
            self.real_target = calc_price_target(params=params, prev_day=prev_day, prev_lags=prev_lags)

            #AMM k
            if prev_day.liq_ratio < params.max_liq_fmcap_ratio:
                self.k = prev_day.k * (1 + self.reward_rate)**2
            else:
                self.k = prev_day.k * (params.max_liq_fmcap_ratio * (1 + self.reward_rate)**2) / (prev_day.liq_ratio)

            # Reserves in
            if prev_day.reserves * (1 - params.max_liq_ratio) < prev_day.liq_usd * params.max_liq_ratio:
                self.reserves_in = (prev_day.liq_usd * params.max_liq_ratio - prev_day.reserves * (1 - params.max_liq_ratio)) / (params.short_cycle / params.cycle_reweights)
            elif prev_day.liq_usd * (1 - params.max_liq_ratio) > prev_day.reserves * params.max_liq_ratio:
                self.reserves_in = -2 * (prev_day.reserves * params.max_liq_ratio - prev_day.liq_usd * (1 - params.max_liq_ratio)) / (params.short_cycle / params.cycle_reweights)
            else:
                self.reserves_in = 0

            # Market dynamics
            self.net_flow = random.uniform(prev_day.liq_usd * prev_day.total_supply, prev_day.liq_usd * prev_day.total_demand) + prev_day.release_capture
            self.market_demand = params.demand_factor * short_sin(self.day, params.short_cycle) * long_sin(self.day, params.long_cycle, params.long_sin_offset)
            self.market_supply = params.supply_factor * short_cos(self.day, params.short_cycle) * long_cos(self.day, params.long_cycle, params.long_cos_offset)
            # Ask Zeus if 60 is indeed == (params.short_cycle / 2) or if it was hardcoded as a magic number
            self.arb_factor = (prev_day.real_target / prev_day.price) ** ((self.day % (params.short_cycle / params.cycle_reweights)) / (params.short_cycle / 2)) - 1
            self.arb_demand = params.demand_factor * self.arb_factor
            self.arb_supply = params.supply_factor * (1 / (1 + self.arb_factor) - 1)

            if self.day >= params.short_cycle:
                if self.day % (params.short_cycle / params.cycle_reweights) == 0:
                    unwind_date = self.day - (2 * self.day % params.short_cycle) - 1
                    self.unwind_demand = prev_arbs[unwind_date][1] * (params.release_capture - 1)
                    self.unwind_supply = prev_arbs[unwind_date][0] * (1 - params.release_capture)
                else:
                    self.unwind_demand = 0
                    self.unwind_supply = 0
            else:
                self.unwind_demand = 0
                self.unwind_supply = 0
            prev_arbs[self.day] = (self.arb_demand, self.arb_supply)

            # Liquidity
            if prev_day.day % (params.short_cycle / params.cycle_reweights) == 0:
                self.liq_usd = (prev_day.real_target * prev_day.k)**(1/2) - self.reserves_in
                self.liq_ohm = self.k / self.liq_usd
                self.price = self.real_target
            else:
                self.liq_usd = prev_day.liq_usd + self.net_flow - self.reserves_in
                self.liq_ohm = self.k / self.liq_usd
                self.price = self.liq_usd / self.liq_ohm

            # Reserves
            self.reserves_out = self.liq_usd - self.net_flow - prev_day.liq_usd
            self.reserves = prev_day.reserves - self.reserves_out
            self.prev_reserves = prev_day.reserves
            if self.day % params.short_cycle == 0:
                self.release_capture = (-1) * self.reserves_in * params.release_capture
            else:
                self.release_capture = 0


        self.treasury = self.liq_usd + self.reserves
        self.mcap = self.supply * self.price
        self.floating_supply = self.supply - self.liq_ohm
        self.floating_mcap = self.floating_supply * self.price
        self.liq_ratio = self.liq_usd / self.reserves
        self.liq_fmcap_ratio = self.liq_usd / self.floating_mcap
        self.net_arb = self.arb_demand + self.arb_supply + self.unwind_demand + self.unwind_supply
        self.total_demand = self.market_demand + self.arb_demand + self.unwind_demand
        self.total_supply = self.market_supply + self.arb_demand + self.unwind_supply
        self.total_net = self.total_demand + self.total_supply


        #only to for reporting purposes (to check calculations)
        prev_lags['price'][1][self.day] = self.price
        prev_lags['target'][1][self.day] = self.real_target
        prev_lags['natural'][1][self.day] = self.natural_target


In [63]:
# Target price controller
def calc_price_target(params:ModelParams, prev_day:Day, prev_lags:Dict[int, Tuple[int, Dict[int, float]]]):
    if params.target_price_function == 'avg_lags':
        if prev_day.day % (params.short_cycle / params.cycle_reweights) == 0:
            s = 0
            lag_keys = set(prev_lags.keys()) - set(['price', 'target', 'natural', 'avg'])
            for key in lag_keys:
                days = prev_lags[key][1].keys()
                s += prev_lags[key][1][max(days)]
            avg_lag = s / len(lag_keys)
            return (prev_day.natural_target + avg_lag) / 2
        else:
            return prev_day.real_target

    elif params.target_price_function == 'price_cycle_avg':
        if prev_day.day % (params.short_cycle / params.cycle_reweights) == 0:
            s = 0
            days = len(prev_lags['price'][1]) - 1
            if days < params.short_cycle / params.cycle_reweights:
                prev_lags['price'][1][days - 2]
            else:
                for i in range(days - int(params.short_cycle / params.cycle_reweights), days):
                    s += prev_lags['price'][1][i]
                avg_lag = s / (params.short_cycle / params.cycle_reweights)
                return avg_lag
        else:
            return prev_day.real_target


def calc_natural_target(params:ModelParams, prev_day:Day):
    if prev_day.day % (params.short_cycle / params.cycle_reweights) == 0:
      if prev_day.day % (params.short_cycle * 4) == 0:
        return ((prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves) + prev_day.real_target) / 2
      else:
        return prev_day.natural_target * prev_day.reserves / prev_day.prev_reserves
    else:
        return prev_day.natural_target


def calc_lag(day:int, params:ModelParams, prev_lags:Dict[int, Tuple[int, Dict[int, float]]], num_days:int=3):
    for key, values in prev_lags.items():
        if key not in ('price', 'target', 'natural', 'avg'):
          lag_days = values[0]
          if day > lag_days:
              if key == 'lag1':
                  if day > params.short_cycle:
                      s = prev_lags['price'][1][day-1]
                      for i in range(1, num_days):
                          s += prev_lags['price'][1][day - (i * lag_days)]
                      prev_lags[key][1][day] = s / num_days
                  else:
                    prev_lags[key][1][day] = values[1][day - 1]
              else:
                prev_lags[key][1][day] = prev_lags['lag1'][1][day - values[0]]
          else:
            prev_lags[key][1][day] = values[1][day - 1]


### __SET SCENARIO PARAMETERS__

In [69]:
# Simulate scenario
params = ModelParams(
    horizon = 780, max_liq_ratio = 0.5, max_liq_fmcap_ratio = 0.2, demand_factor = 0.01, supply_factor = -0.01,
    initial_supply = 11000000, initial_reserves = 244000000, initial_liq_usd = 80000000,
    initial_price = 36, initial_target = 40, target_price_function = 'price_cycle_avg', arb_factor = 0, release_capture = 0.333,
    short_cycle = 120, cycle_reweights = 1, long_cycle = 1460, long_sin_offset = 1.125, long_cos_offset = 0.125
)

lags = {
    'price': (0, {1: params.initial_price}), 'target': (0, {1: params.initial_target}), 'natural': (0, {1: params.initial_target}), 'avg': (0, {1: params.initial_target}),
    'lag1': (30, {1: params.initial_target}), 'lag2': (90, {1: params.initial_target}), 'lag3': (180, {1: params.initial_target}), 'lag4': (360, {1: params.initial_target})
}

arbs = {}

simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags)}

for i in range (2, params.horizon):
    simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'])
    calc_lag(day=i, params=params, prev_lags=lags)

ZeroDivisionError: float division by zero

### __PLOT RESULTS__

In [65]:
# Protocol variables
df = pd.DataFrame(columns = ['NetFlow', 'Price', 'RealTarget', 'NaturalTarget', 'LiqUSD', 'LiqOHM', 'Reserves', 'ReservesIN', 'ReservesOUT', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio']) 
for day, data in simulation.items():
    df.loc[day] = [data.net_flow, data.price, data.real_target, data.natural_target, data.liq_usd, data.liq_ohm, data.reserves, data.reserves_in, data.reserves_out, data.treasury, data.supply, data.mcap, data.floating_supply, data.floating_mcap, data.liq_ratio]


In [66]:
# Plot batch1
for col in df.columns:
    if col in ('NetFlow', 'LiqUSD', 'LiqOHM', 'LiqRatio'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        if col == 'LiqRatio':
            fig.add_hline(y=0.2, line_width=1, line_dash="dash", line_color="grey")
        fig.show()

In [ ]:
# Plot batch2
for col in df.columns:
    if col in ('Reserves', 'ReservesIN', 'ReservesOUT', 'Treasury'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        fig.show()

In [67]:
# Plot batch3
for col in df.columns:
    if col in ('Supply', 'FloatingSupply', 'MCap', 'FloatingMCap'):
        fig = df.plot(x=df.index, y=df[col])
        fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, params.horizon, params.short_cycle):
            fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
        fig.show()

In [68]:
# Plot batch4
df[['Price','RealTarget', 'NaturalTarget']].plot()

In [ ]:
# Market dynamics variables
market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'ArbFactor', 'ArbDemand', 'ArbSupply', 'UnwindDemand', 'UnwindSupply', 'TotalDemand', 'TotalSupply', 'NetTotal', 'NetArb']) 
for day, data in simulation.items():
    market_df.loc[day] = [data.market_demand, data.market_supply, data.arb_factor, data.arb_demand, data.arb_supply, data.unwind_demand, data.unwind_supply, data.total_demand, data.total_supply, data.total_net, data.net_arb]

for col in market_df.columns:
    fig = market_df.plot(x=df.index, y=market_df[col])
    fig.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
    for i in range (params.short_cycle, params.horizon, params.short_cycle):
        fig.add_vline(x=i, line_width=0.5, line_dash="dot", line_color="grey")
    fig.show()

In [ ]:
df2 = pd.DataFrame(columns = ['shortSin', 'shortCos', 'longSin', 'longCos']) 
for i in range (2, 2*params.horizon):
    df2.loc[f'day{i}'] = [short_sin(i, params.short_cycle), short_cos(i, params.short_cycle), long_sin(i, params.long_cycle, params.long_sin_offset), long_cos(i, params.long_cycle, params.long_cos_offset)]

df2.plot(y=df2.columns)

In [ ]:
# Variables related to price controller
price_target_df = pd.DataFrame(columns = ['Price', 'Target', 'Natural', 'Avg', 'Lag1', 'Lag2', 'Lag3', 'Lag4']) 
for key in lags.keys():
    for day, p in lags[key][1].items():
      price_target_df.at[day, key.capitalize()] = p

price_target_df.plot()